In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Sequential,Model
import re

Using TensorFlow backend.


In [10]:
train = pd.read_csv("Train.csv")
train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [11]:
train = train.values
answers = {}
distractors = {}
count = 0
for x in range(train.shape[0]):
    answers[train[x][0]] = train[x][1]
    a=[]
    for y in train[x][2].split(", "):
        a.append(str(y[1:-1]))
    distractors[train[x][0]] = a
    count = count+1

In [12]:
distractors["Meals can be served"]

['outside the room at 3:00 p. m.',
 'in the dining - room at 6:00 p. m.',
 'in the dining - room from 7:30 a. m. to 9:15 p. m.']

In [13]:
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub("[^a-z0-9]+"," " , sentence)
    sentence = sentence.split()

    sentence = [s for s in sentence if((len(s)>1) or (re.match("[0-9]+",s) is not None))]
    sentence = " ".join(sentence)

    return sentence

In [14]:
# Clean all the captions
a={}
d={}
for key , dist_list in distractors.items():
    for i in range(len(dist_list)):
        dist_list[i] = clean_text(dist_list[i])
    answer=clean_text(answers[key])
    key=clean_text(key)
    a[key]=answer
    d[key]=dist_list
answers=a
distractors=d

In [15]:
distractors["meals can be served"]

['outside the room at 3 00',
 'in the dining room at 6 00',
 'in the dining room from 7 30 to 9 15']

In [16]:
with open("answers.txt","w") as f:
    f.write(str(answers))
with open("distractors.txt","w") as f:
    f.write(str(distractors))

In [18]:
vocab = set()
for key in answers.keys():
    [vocab.update(key.split())]
    [vocab.update(answers[key].split())]
    [vocab.update(sentence.split()) for sentence in distractors[key]]

    
print(len(vocab))

21459


In [19]:
total = []
for key in answers.keys():
    [total.append(i) for i in key.split()]
    [total.append(i) for i in answers[key].split()]
    [total.append(i) for des in distractors[key] for i in des.split()]

print(len(total))

718584


In [20]:
import collections
counter = collections.Counter(total)
freq_cnt = dict(counter)

sorted_freq_cnt = sorted(freq_cnt.items(),reverse = True,key = lambda x:x[1])

threshold =10
sorted_freq_cnt = [x for x in sorted_freq_cnt if x[1]>threshold]
total_words = [x[0] for x in sorted_freq_cnt]
print(len(sorted_freq_cnt))

4723


In [21]:
train_distractors = {}
for key in distractors.keys():
    train_distractors[key] = []
    for dist in distractors[key]:
        dist_to_append = "StartSeq " + dist + " EndSeq"
        train_distractors[key].append(dist_to_append)
        
print(train_distractors["meals can be served"])

['StartSeq outside the room at 3 00 EndSeq', 'StartSeq in the dining room at 6 00 EndSeq', 'StartSeq in the dining room from 7 30 to 9 15 EndSeq']


In [22]:
len(total_words)

4723

In [23]:
word_to_idx = {}
idx_to_word = {}

for i,word in enumerate(total_words):
    word_to_idx[word] = i+1
    idx_to_word[i+1] = word
    
len(word_to_idx)

4723

In [24]:
word_to_idx["StartSeq"]=4724
idx_to_word[4724] = "StartSeq"

word_to_idx["EndSeq"]=4725
idx_to_word[4725] = "EndSeq"

vocab_size = len(word_to_idx)

vocab_size= vocab_size+1

In [25]:
vocab_size

4726

In [26]:
max_len=0
for key in train_distractors.keys():
    for dist in train_distractors[key]:
        max_len = max(max_len,len(dist.split()))
print(max_len)

30


In [27]:
max_q=0
for key in train_distractors.keys():
    max_q = max(max_q,len(key.split()))
print(max_q)

48


In [28]:
max_a = 0
for key in answers.keys():
    max_a = max(max_a,len(answers[key].split()))
print(max_a)

101


In [ ]:
def data_generator(train_distractors,answers,word_to_idx,max_len,batch_size):
    X1,X2,X3,y = [],[],[],[]

    n=0
    while True:
        for key,dist_list in train_distractors.items():
            n+=1

            question = key
            answer = answers[key]


            seqq = [word_to_idx[wordQ] for wordQ in question.split() if wordQ in word_to_idx]
            question= pad_sequences([seqq],maxlen=max_q,value=0,padding='post')[0]


            seqa = [word_to_idx[wordA] for wordA in answer.split() if wordA in word_to_idx]
            answer = pad_sequences([seqa],maxlen=max_a,value=0,padding='post')[0]

            for dist in dist_list:
                seq = [word_to_idx[word] for word in dist.split() if word in word_to_idx]
                for i in range(1,len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]

                    xi = pad_sequences([xi],maxlen=max_len,value = 0,padding='post')[0] 
                    yi = to_categorical([yi],num_classes = vocab_size)[0]



                    X1.append(question)
                    X2.append(answer)
                    X3.append(xi)
                    y.append(yi)
                if n==batch_size:
                yield[[np.array(X1),np.array(X2),np.array(X3)],np.array(y)]
                X1,X2,X3,y = [],[],[],[]
                n=0